In [1]:
# If you've git cloned this repo, run this cell to add `geostructures` to your python path
import sys
sys.path.insert(0, '../')

In [2]:
from geostructures import *
from geostructures.collections import FeatureCollection, Track
from datetime import datetime, timedelta
from geostructures.geohash import H3Hasher
from geochron import convert

import pandas as pd
hasher = H3Hasher(resolution=10)

# To create a FeatureCollection, simply pass it a list of geoshapes
fcol = FeatureCollection(
    [
        GeoCircle(Coordinate(-0.118092, 51.509865), 500, dt=datetime(2020, 1, 1, 9, 5)),
        GeoEllipse(Coordinate(-0.118072, 51.502365), 2000, 400, 80, dt=datetime(2020, 1, 1, 9, 5)),
        GeoCircle(Coordinate(-0.141092, 51.529865), 500, dt=datetime(2020, 1, 1, 9, 5)),
        GeoRing(Coordinate(-0.116092, 51.519865), inner_radius=800, outer_radius=1000, dt=datetime(2020, 1, 1, 10, 5)),
        GeoRing(Coordinate(-0.101092, 51.514865), inner_radius=300, outer_radius=500, angle_min=60, angle_max=190, dt=datetime(2020, 1, 2, 11, 5)),
    ]
)



[WARNING] geostructures: Datetime does not contain timezone information; Zulu/UTC time assumed. (this warning will not repeat)


In [111]:
import datetime as dt
from geochron.time_slicing import get_timestamp_intervals, time_slice_track
from geostructures import Coordinate, GeoPoint
from geostructures.collections import  Track

track = Track(
    [
        GeoPoint(Coordinate(-0.104154, 51.511920), dt=dt.datetime(2020, 1, 1, 8, 5)),
        GeoPoint(Coordinate(-0.096533, 51.511903), dt=dt.datetime(2020, 1, 1, 9, 23)),
        GeoPoint(Coordinate(-0.083765, 51.514423), dt=dt.datetime(2020, 1, 1, 9, 44)),
        GeoPoint(Coordinate(-0.087478, 51.508595), dt=dt.datetime(2020, 1, 1, 10, 5)),
        GeoPoint(Coordinate(-0.087478, 51.508595), dt=dt.datetime(2020, 1, 1, 11, 7)),
    ]
)

In [113]:
timestamps = get_timestamp_intervals(track,1)

In [114]:
timestamps

[datetime.datetime(2020, 1, 1, 9, 5, tzinfo=datetime.timezone.utc),
 datetime.datetime(2020, 1, 1, 10, 5, tzinfo=datetime.timezone.utc),
 datetime.datetime(2020, 1, 1, 11, 5, tzinfo=datetime.timezone.utc),
 datetime.datetime(2020, 1, 1, 12, 5, 1, tzinfo=datetime.timezone.utc)]

In [115]:
list1 = time_slice_track(track,timestamps)

In [116]:
list1

[<Track with 1 shapes from 2020-01-01T08:05:00+00:00 - 2020-01-01T08:05:00+00:00>,
 <Track with 2 shapes from 2020-01-01T09:23:00+00:00 - 2020-01-01T09:44:00+00:00>,
 <Track with 1 shapes from 2020-01-01T10:05:00+00:00 - 2020-01-01T10:05:00+00:00>,
 <Track with 1 shapes from 2020-01-01T11:07:00+00:00 - 2020-01-01T11:07:00+00:00>]

In [117]:
from typing import List, Callable
def hash_tracks_into_netdf(track_list: List, timestamps: List, hash_func: Callable):
    """
    Converts a list of tracks into a pandas dataframe suitable for chron net creation
    using a specified hashing function with intervals reflected
    in a corresponding timestamp list 
    
    Args:
        track_list: a list of tracks broken down by equal intervals

        timestamps: a list of corresponding timestamps

        hash_func: the hashing function

    Returns:
        A pandas dataframe
    """
    interval_start = track_list[0].start
    master_list = []
    for track,timestamp  in zip(track_list, timestamps):
        hashmap = hash_func(track)
        start_string= interval_start.strftime("%Y-%m-%d %H:%M:%S")
        end_string= timestamp.strftime("%Y-%m-%d %H:%M:%S")
        interval = start_string + ", " + end_string
        hexes = list(hashmap.keys())
        counts = list(hashmap.values())
        hex_list = [element for element, count in zip(hexes, counts) for _ in range(count)]
        tuples_list = [(item, interval) for item in hex_list]
        master_list += tuples_list
        interval_start = timestamp
        

    df = pd.DataFrame(master_list, columns=['cell', 'time'])
    
    return df

In [ ]:
# Define your list and string
your_list = [1, 2, 3, 4]
your_string = 'abcd'

# Create a list of tuples
tuples_list = [(item, your_string) for item in your_list]

print(tuples_list)


In [118]:

df = hash_tracks_into_netdf(list1, timestamps, hasher.hash_collection)

In [119]:
df

,cell,time
0,8a194ad32167fff,"2020-01-01 08:05:00, 2020-01-01 09:05:00"
1,8a194ad32b07fff,"2020-01-01 09:05:00, 2020-01-01 10:05:00"
2,8a194ad3056ffff,"2020-01-01 09:05:00, 2020-01-01 10:05:00"
3,8a194ad3078ffff,"2020-01-01 10:05:00, 2020-01-01 11:05:00"
4,8a194ad3078ffff,"2020-01-01 11:05:00, 2020-01-01 12:05:01"


In [94]:
import itertools
import networkx as nx
import numpy as np
def chronnet_create(df: pd.DataFrame, self_loops: bool, mode= str):
    """
    Converts a properly formatted pandas dataframe with the columns
    of cell and time to a networkx network. 
    
    Args:
        df: a pandas dataframe withe two columns cell and time

        self_loops: whether self loops are included in the network

        mode: whether the network is directed or undirected

    Returns:
        A networkx network
    """
    time_seq = sorted(np.unique(df['time']))
    if len(time_seq) < 2: # pragma: no cover
        print("The total time interval in the dataset should be larger than two.")
    links = pd.DataFrame()
    cells_before=[]
    cells_after=[]
    weights = []
    for i in range(len(time_seq)-1):
        time1 = df[df.time == time_seq[i]]['cell'].tolist()
        time2 = df[df.time == time_seq[i+1]]['cell'].tolist()
        connections = list(itertools.product(time1, time2))
        cells_before.extend([connection[0] for connection in connections])
        cells_after.extend([connection[1] for connection in connections])
        weights.extend([1]*len(connections))
    links['from'] = cells_before
    links['to'] = cells_after
    links['weight'] = weights
    links = links.sort_values(by=['from', 'to']).reset_index().drop(columns = ['index'])
    links = links.groupby(['from', 'to'], as_index=False)['weight'].sum()
    net = nx.DiGraph()
    if len(links) !=0:
        net.add_nodes_from(np.unique(df['cell']))
        edgelist = []
        for index, rows in links.iterrows():
            edgelist.append(tuple([rows['from'], rows['to'], rows['weight']]))
        net.add_weighted_edges_from(edgelist)
        if self_loops == False:
            net.remove_edges_from(list(nx.selfloop_edges(net)))
        if mode == 'undirected':
            net = net.to_undirected()
    else: # pragma: no cover
        print("Empty graph returned.")
    
    return connections

In [95]:
network = chronnet_create(df, True)

In [96]:
network

[('8a194ad32167fff', '8a194ad32b07fff'),
 ('8a194ad32167fff', '8a194ad3056ffff'),
 ('8a194ad32167fff', '8a194ad3078ffff'),
 ('8a194ad32167fff', '8a194ad3078ffff')]

In [84]:
network.edges(data=True)

OutEdgeDataView([('8a194ad32167fff', '8a194ad3056ffff', {'weight': 1}), ('8a194ad32167fff', '8a194ad3078ffff', {'weight': 2}), ('8a194ad32167fff', '8a194ad32b07fff', {'weight': 1})])

In [83]:
import networkx as nx 
list(nx.selfloop_edges(network))

[]

In [ ]:
geopandasdf= fcol.to_geopandas()

In [ ]:
geopandasdf

,datetime_end,datetime_start,geometry
0,2020-01-01 09:05:00+00:00,2020-01-01 09:05:00+00:00,"POLYGON ((-0.11809 51.51436, -0.11935 51.51429..."
1,2020-01-01 09:05:00+00:00,2020-01-01 09:05:00+00:00,"POLYGON ((-0.08961 51.50548, -0.09021 51.50605..."
2,2020-01-01 09:05:00+00:00,2020-01-01 09:05:00+00:00,"POLYGON ((-0.14109 51.53436, -0.14235 51.53429..."
3,2020-01-01 10:05:00+00:00,2020-01-01 10:05:00+00:00,"POLYGON ((-0.11609 51.52886, -0.11860 51.52872..."
4,2020-01-02 11:05:00+00:00,2020-01-02 11:05:00+00:00,"POLYGON ((-0.10235 51.51044, -0.10109 51.51037..."


In [ ]:
test= convert(fcol, "chronnet", 1)

In [ ]:
test

In [ ]:
test

,interval,8a194ad320affff,8a194ad32787fff,8a194ad324e7fff,8a194ad3246ffff,8a195da49b6ffff,8a194ad32467fff,8a194ad3208ffff,8a194ad32617fff,8a194ad32707fff,...,8a194ad30487fff,8a194ad304b7fff,8a194ad32b17fff,8a194ad32b9ffff,8a194ad32b37fff,8a194ad32b8ffff,8a194ad32b87fff,8a194ad3049ffff,start_time,end_time
0,"2020-01-01 09:05:00, 2020-01-01 10:05:00",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 09:05:00,2020-01-01 10:05:00
1,"2020-01-01 10:05:00, 2020-01-01 11:05:00",NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 10:05:00,2020-01-01 11:05:00
2,"2020-01-01 11:05:00, 2020-01-01 12:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 11:05:00,2020-01-01 12:05:00
3,"2020-01-01 12:05:00, 2020-01-01 13:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 12:05:00,2020-01-01 13:05:00
4,"2020-01-01 13:05:00, 2020-01-01 14:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 13:05:00,2020-01-01 14:05:00
5,"2020-01-01 14:05:00, 2020-01-01 15:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 14:05:00,2020-01-01 15:05:00
6,"2020-01-01 15:05:00, 2020-01-01 16:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 15:05:00,2020-01-01 16:05:00
7,"2020-01-01 16:05:00, 2020-01-01 17:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 16:05:00,2020-01-01 17:05:00
8,"2020-01-01 17:05:00, 2020-01-01 18:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 17:05:00,2020-01-01 18:05:00
9,"2020-01-01 18:05:00, 2020-01-01 19:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 18:05:00,2020-01-01 19:05:00


In [ ]:
track = Track(fcol.geoshapes)

In [ ]:
track.end

datetime.datetime(2020, 1, 2, 11, 5, tzinfo=datetime.timezone.utc)

In [ ]:
from datetime import datetime, timedelta

# list of datetime objects
datetime_list = [datetime(2022, 1, 1), datetime(2022, 1, 2), datetime(2022, 1, 3)]
print("Original list: ", datetime_list)

# change the last value
datetime_list[-1] = datetime_list[-1] + timedelta(seconds=1)
print("After changing the last value: ", datetime_list)


Original list:  [datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 1, 2, 0, 0), datetime.datetime(2022, 1, 3, 0, 0)]
After changing the last value:  [datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 1, 2, 0, 0), datetime.datetime(2022, 1, 3, 0, 0, 1)]


In [ ]:
def get_timestamp_intervals(track: Track, hour_interval: int):
    """
    gets the timestamps for a Track partitioned by a specified length
    of time in hours  
    
    Args:
        track: the target Track

        hour_interval= the length in hours of the desired interval

    Returns:
        A list of timestamps starting from an interval from the start
    """
    start_time = track.start
    end_time = track.end
    timestamps = []

    # get timestamp intervals
    date_x = start_time
    while date_x < end_time:
        date_x += timedelta(hours = hour_interval)
        timestamps.append(date_x)
    
    # change the last value to be inclusive 
    timestamps[-1] = timestamps[-1] + timedelta(seconds=1)
    
    
    return timestamps

In [ ]:
def time_slice_track(track: Track, timestamps):
    """
    Slices a Track into several tracks that are partitioned by a list
    of time stamps  
    
    Args:
        track: the target Track

        timestamps= a list of timestamps

    Returns:
        A list of tracks
    """
    start_time = track.start
    sliced_tracks = []

    
    for timestamp in timestamps:
        sliced_track = track[start_time: timestamp]
        start_time = timestamp
        sliced_tracks.append(sliced_track)
    
    return sliced_tracks

In [ ]:
def hash_tracks_into_df(track_list , timestamps, hash_func):
    """
    Converts a list of tracks into a pandas dataframe using
    a specified hashing function with intervals reflected
    in a corresponding timestamp list 
    
    Args:
        track_list: a list of tracks broken down by equal intervals

        timestamps: a list of corresponding timestamps

        hash_func: the hashing function

    Returns:
        A pandas dataframe
    """
    interval_start = track_list[0].start
    row_list = []
    for track,timestamp  in zip(track_list, timestamps):
        hashmap = hash_func(track)
        start_string= interval_start.strftime("%Y-%m-%d %H:%M:%S")
        end_string= timestamp.strftime("%Y-%m-%d %H:%M:%S")
        interval = start_string + ", " + end_string
        new_row = pd.Series(data=hashmap, name=interval)
        row_list.append(new_row)
        interval_start = timestamp
    
    df = pd.DataFrame(row_list)
    
    return df

In [ ]:
def hash_tracks_into_netdf(track_list , timestamps, hash_func):
    """
    Converts a list of tracks into a pandas dataframe using
    a specified hashing function with intervals reflected
    in a corresponding timestamp list 
    
    Args:
        track_list: a list of tracks broken down by equal intervals

        timestamps: a list of corresponding timestamps

        hash_func: the hashing function

    Returns:
        A pandas dataframe
    """
    interval_start = track_list[0].start
    master_hashmap = {}
    for track,timestamp  in zip(track_list, timestamps):
        hashmap = hash_func(track)
        start_string= interval_start.strftime("%Y-%m-%d %H:%M:%S")
        end_string= timestamp.strftime("%Y-%m-%d %H:%M:%S")
        interval = start_string + ", " + end_string
        hashmap = {key: interval for key in hashmap}
        interval_start = timestamp
        master_hashmap.update(hashmap)

    df = pd.DataFrame(list(master_hashmap.items()), columns=['cell', 'time'])
    return df

In [ ]:
timestamps = get_timestamp_intervals(track, 1)

In [ ]:
test=time_slice_track(track, timestamps)

In [ ]:
df = hash_tracks_into_netdf(test, timestamps, hasher.hash_collection)

In [ ]:
df

,cell,time
0,8a194ad3201ffff,"2020-01-01 09:05:00, 2020-01-01 10:05:00"
1,8a194ad3208ffff,"2020-01-01 09:05:00, 2020-01-01 10:05:00"
2,8a194ad32717fff,"2020-01-01 09:05:00, 2020-01-01 10:05:00"
3,8a194ad3245ffff,"2020-01-01 09:05:00, 2020-01-01 10:05:00"
4,8a194ad32797fff,"2020-01-01 09:05:00, 2020-01-01 10:05:00"
...,...,...
379,8a194ad32b87fff,"2020-01-02 10:05:00, 2020-01-02 11:05:01"
380,8a194ad32b1ffff,"2020-01-02 10:05:00, 2020-01-02 11:05:01"
381,8a194ad32b17fff,"2020-01-02 10:05:00, 2020-01-02 11:05:01"
382,8a194ad32b8ffff,"2020-01-02 10:05:00, 2020-01-02 11:05:01"


In [ ]:
df = df.reset_index(names='interval')

In [ ]:

df[['start_time', 'end_time']] = df['interval'].str.split(',', expand=True)


In [ ]:
df

,interval,8a194ad3201ffff,8a194ad3208ffff,8a194ad32717fff,8a194ad3245ffff,8a194ad32797fff,8a194ad320affff,8a194ad3254ffff,8a194ad3218ffff,8a194ad32607fff,...,8a194ad30497fff,8a194ad32b9ffff,8a194ad32b27fff,8a194ad32b87fff,8a194ad32b1ffff,8a194ad32b17fff,8a194ad32b8ffff,8a194ad32b37fff,start_time,end_time
0,"2020-01-01 09:05:00, 2020-01-01 10:05:00",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 09:05:00,2020-01-01 10:05:00
1,"2020-01-01 10:05:00, 2020-01-01 11:05:00",NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 10:05:00,2020-01-01 11:05:00
2,"2020-01-01 11:05:00, 2020-01-01 12:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 11:05:00,2020-01-01 12:05:00
3,"2020-01-01 12:05:00, 2020-01-01 13:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 12:05:00,2020-01-01 13:05:00
4,"2020-01-01 13:05:00, 2020-01-01 14:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 13:05:00,2020-01-01 14:05:00
5,"2020-01-01 14:05:00, 2020-01-01 15:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 14:05:00,2020-01-01 15:05:00
6,"2020-01-01 15:05:00, 2020-01-01 16:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 15:05:00,2020-01-01 16:05:00
7,"2020-01-01 16:05:00, 2020-01-01 17:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 16:05:00,2020-01-01 17:05:00
8,"2020-01-01 17:05:00, 2020-01-01 18:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 17:05:00,2020-01-01 18:05:00
9,"2020-01-01 18:05:00, 2020-01-01 19:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 18:05:00,2020-01-01 19:05:00


In [ ]:
from datetime import datetime, timedelta
date_a = datetime(2016, 8, 9, 1, 24, 30, 993352)
date_b = datetime(2016, 8, 9, 7, 24, 30, 993352)

delta = timedelta(hours=1)
five_min_timestamps = []

date_x = date_a
while date_x < date_b:
    date_x += delta
    five_min_timestamps.append(date_x)

print(five_min_timestamps)

[datetime.datetime(2016, 8, 9, 2, 24, 30, 993352), datetime.datetime(2016, 8, 9, 3, 24, 30, 993352), datetime.datetime(2016, 8, 9, 4, 24, 30, 993352), datetime.datetime(2016, 8, 9, 5, 24, 30, 993352), datetime.datetime(2016, 8, 9, 6, 24, 30, 993352), datetime.datetime(2016, 8, 9, 7, 24, 30, 993352)]
